# 08. Core photo preparation U1474
This step is necessary because the photos are a bit too large to play with in these notebooks, in the 1-5 MB range. Many of these notebooks include lots of photos of cores, so if they are too big, the memory on my little MacBook Air gets overwhelmed. Also, I rotate them in this step so if all I want to do is make a simple plot with the core, I won't need to rotate them again. 

## Setup
### Import Modules

In [1]:
import pandas as pd
from IPython.display import clear_output
import time
import cv2 as cv

### Set Paths

In [2]:
data_path='/Users/danielbabin/GitHub/Green_Bands/Data/'
lowres='/Volumes/SanDisk.Data/IODPU1474CorePhotos/LowRes/Raw/'
medres='/Volumes/SanDisk.Data/IODPU1474CorePhotos/MedRes/'

### Import Splice Info

In [3]:
splice=pd.read_csv(data_path+'SpliceTables/spliceConstructionE361.csv')
splice=splice[splice['Site']=='U1474']
splice['CoreLabel']=(splice['Site']+splice['Hole']+'_'+splice['Core'].map(str))

section_list=pd.read_csv(data_path+'SectionSummaries/sectionSummaryU1474.csv')
section_list['SectionLabel']=(section_list['Site']+section_list['Hole']+'_'+
                              section_list['Core'].map(str)+section_list['Type']+
                              section_list['Sect'].map(str))
section_list=section_list.rename(columns={'Top depth CCSF-361-U1474-ABDEF-20160220 (m)':'Top depth CCSF (m)',
                                          'Bottom depth CCSF-361-U1474-ABDEF-20160220 (m)':'Bottom depth CCSF (m)'})

### Create List of Splice Sections

In [4]:
splice_sections=[]
for i in splice.index:
    for s in range(splice.loc[i,'Top section'],splice.loc[i,'Bottom section']+1):
        entry=(splice.loc[i,'Site']+splice.loc[i,'Hole']+'_'+splice.loc[i,'Core'].astype(str)+
                       splice.loc[i,'Type']+str(s))
        splice_sections.append(entry)

In [5]:
pd.DataFrame(splice_sections,columns=['sectionLabel']).to_csv(data_path+
                                                              'SectionSummaries/spliceSectionListU1474.csv',
                                                              index=False)

In [6]:
sections=section_list[(section_list['Top depth CCSF (m)']<205)&
                      (section_list['Sect']!='CC')]['SectionLabel']

### Import Metadata for Core Photos

In [7]:
cpmd=pd.read_csv(data_path+'U1474/corePhotosMetadataU1474.csv')
cpmd['SectionLabel']=cpmd['Site']+cpmd['Hole']+'_'+cpmd['Core'].map(str)+cpmd['Type']+cpmd['Sect'].map(str)
iddrop=cpmd[(cpmd['SectionLabel']=='U1474F_6H2')&(cpmd['A/W']=='W')].index
cpmd=cpmd.drop(iddrop)
cpmd['Cropped image filename (JPEG)']=cpmd['Cropped image filename'].str[:-4]+'.jpeg'
cpmd.set_index('SectionLabel',inplace=True)
cpmd=cpmd[cpmd['Display status (T/F)']=='T']

## Resize and Rotate Photos
I write everything in functions because it saves memory.

### Path Finding Function
The IODP download on Zenodo breaks the photos up by their hole, so it's a little extra effort to find the path to the right photo.

In [8]:
def path_finder(section):
    if section[5]=='A':
        path='/Volumes/SanDisk.Data/IODPU1474CorePhotos/Raw/361-U1474A-LSIMG/'
    elif section[5]=='B':
        path='/Volumes/SanDisk.Data/IODPU1474CorePhotos/Raw/361-U1474B-LSIMG/'
    elif section[5]=='C':
        path='/Volumes/SanDisk.Data/IODPU1474CorePhotos/Raw/361-U1474C-LSIMG/'
    elif section[5]=='D':
        path='/Volumes/SanDisk.Data/IODPU1474CorePhotos/Raw/361-U1474D-LSIMG/'
    elif section[5]=='E':
        path='/Volumes/SanDisk.Data/IODPU1474CorePhotos/Raw/361-U1474E-LSIMG/'
    else:
        path='/Volumes/SanDisk.Data/IODPU1474CorePhotos/Raw/361-U1474F-LSIMG/'
    return path

### Function for reducing image size, rotating it, and exporting it
For this, I use the package OpenCV's resize and rotate methods.

In [9]:
def shrink_rotate_write(section,shrinkfactor):
    path=path_finder(section)
    raw = cv.imread(path+cpmd.loc[section,'Cropped image filename'][:-4]+'-LSIMG.jpg')
    size=raw.shape
    newsize=tuple((int(size[1]/shrinkfactor),int(size[0]/shrinkfactor)))
    sraw=cv.resize(raw,newsize,interpolation = cv.INTER_AREA)
    srotraw=cv.rotate(sraw, cv.ROTATE_90_COUNTERCLOCKWISE)
    cv.imwrite(lowres+cpmd.loc[section,'Cropped image filename'],srotraw)

In [14]:
start=time.time()
for i,s in enumerate(sections):
    clear_output(wait=True)
    shrink_rotate_write(s,15)
    stop=time.time()-start
    if i/len(sections)*100<2:
        expected_time='Calculating...'
    else:
        expected_time=round((stop-0)/(i/len(sections))/60,2)
    print('Current progress: ',round(i/len(sections)*100,2),'%')
    print('Current run time: ',round((stop-0)/60,2),' minutes')
    print('Expected run time: ',expected_time,' minutes')

Current progress:  99.81 %
Current run time:  5.07  minutes
Expected run time:  5.08  minutes


## Collect Splice Sections
This takes all of the sections in the splice and puts the full size photo in a new directory. Having only the splice sections there makes some steps a little easier.

In [15]:
section_path='/Volumes/SanDisk.Data/IODPU1474CorePhotos/SpliceSections/'

In [16]:
def rotate_photo(section):
    path=path_finder(section)
    raw = cv.imread(path+cpmd.loc[section,'Cropped image filename'][:-4]+'-LSIMG.jpg')
    rotraw=cv.rotate(raw, cv.ROTATE_90_COUNTERCLOCKWISE)
    cv.imwrite(section_path+cpmd.loc[section,'Cropped image filename'],rotraw)

In [17]:
for i,s in enumerate(sections):
    clear_output(wait=True)
    rotate_photo(s)
    print(i,'/',len(sections))

534 / 535


## Splice Sections Med Res

In [21]:
splice_sections=pd.read_csv(data_path+'SectionSummaries/spliceSectionListU1474.csv')['sectionLabel']

In [22]:
def shrink_rotate_write(section,shrinkfactor):
    path=path_finder(section)
    raw = cv.imread(path+cpmd.loc[section,'Cropped image filename'][:-4]+'-LSIMG.jpg')
    size=raw.shape
    newsize=tuple((int(size[1]/shrinkfactor),int(size[0]/shrinkfactor)))
    sraw=cv.resize(raw,newsize,interpolation = cv.INTER_AREA)
    srotraw=cv.rotate(sraw, cv.ROTATE_90_COUNTERCLOCKWISE)
    cv.imwrite(medres+cpmd.loc[section,'Cropped image filename'],srotraw)

In [24]:
start=time.time()
for i,s in enumerate(splice_sections):
    clear_output(wait=True)
    shrink_rotate_write(s,4)
    stop=time.time()-start
    if i/len(sections)*100<2:
        expected_time='Calculating...'
    else:
        expected_time=round((stop-0)/(i/len(splice_sections))/60,2)
    print('Current progress: ',round(i/len(splice_sections)*100,2),'%')
    print('Current run time: ',round((stop-0)/60,2),' minutes')
    print('Expected run time: ',expected_time,' minutes')

Current progress:  99.54 %
Current run time:  0.88  minutes
Expected run time:  0.88  minutes
